In [ ]:
# %%
from scipy import io as sio
import numpy as np
from pydiffmap import diffusion_map
from matplotlib import pyplot as plt
from sklearn import manifold
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
# %%
data = sio.loadmat('face.mat')
raw_data = data['Y']
imgs = raw_data.reshape(-1, 33).transpose(1, 0).astype(np.float)
gt=np.array([10, 21, 23, 15, 11,  5,  8, 33,  1, 14, 20, 13,  2, 27, 31, 12,  9, 6,  3, 30, 16, 17, 26, 22, 32, 18, 24,  7, 25, 28, 29,  4, 19]) #ground truth

# %% ISOMAP
isomap = Isomap(n_neighbors=5,n_components=2)
ev = isomap.fit_transform(imgs)
n=np.arange(33)
fig,ax=plt.subplots()
ax.scatter(ev[:, 0], ev[:, 1])
ax.title('ISOMAP')
for i,txt in enumerate(n):
    ax.annotate(txt,(ev[i,0],ev[i,1]))
plt.show()

rank = ev[:, 0].argsort()
ev_sort=ev[np.argsort(ev[:,0])]
print(ev[:, 0])
print(rank + 1)
plt.figure(figsize=[12, 5])
for i in range(33):
    plt.subplot(3, 11, i + 1)
    plt.imshow(raw_data[:, :, rank[i]],cmap='gray')
    plt.axis('off')
plt.show()



# %% LLE
lle = LocallyLinearEmbedding(n_neighbors=5,n_components=2)
ev = lle.fit_transform(imgs)
n=np.arange(33)
fig,ax=plt.subplots()
ax.scatter(ev[:, 0], ev[:, 1])
ax.title('LLE')
for i,txt in enumerate(n):
    ax.annotate(txt,(ev[i,0],ev[i,1]))
plt.show()


rank = ev[:, 0].argsort()
ev_sort=ev[np.argsort(ev[:,0])]
print(ev[:, 0])
print(rank + 1)
plt.figure(figsize=[12, 5])
for i in range(33):
    plt.subplot(3, 11, i + 1)
    plt.imshow(raw_data[:, :, rank[i]],cmap='gray')
    plt.axis('off')
plt.show()



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# ISOMAP Scatter
plt.figure(figsize = (20,10))
plt.plot()
ax = plt.gca()
ax.set_ylim(-2500, 3500)
ax.set_xlim(-8000, 10000)

imgplot = [None] * 33
for i in range(33):
    tx, ty = ev_sort[i,:]
    ax.imshow(raw_data[:, :, rank[i]], extent=(tx, tx+460, ty, ty + 460),cmap="gray")
ax.scatter(ev[:, 0], ev[:, 1],s=10)
plt.title('ISOMAP')
plt.show()


# LLE Scatter
plt.figure(figsize = (20,10))
plt.plot()
ax = plt.gca()
ax.set_ylim(-0.4, 0.5)
ax.set_xlim(-0.4, 0.4)
#plt.yticks(np.arange(-8000, 10000, 2000)

imgplot = [None] * 33
for i in range(33):
    tx, ty = ev_sort[i,:]
    ax.imshow(raw_data[:, :, rank[i]], aspect='equal',extent=(tx, tx+0.04, ty, ty + 0.04),cmap="gray")
ax.scatter(ev[:, 0], ev[:, 1],s=10)
plt.title('LLE')
plt.show()


# Parameter Test

# %%
from scipy import io as sio
import numpy as np
from matplotlib import pyplot as plt
from sklearn import manifold
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
data = sio.loadmat('face.mat')
raw_data = data['Y']
imgs = raw_data.reshape(-1, 33).transpose(1, 0).astype(np.float)
gt = np.asarray([9,13,19,32,6,18,28,7,17,1,5,16,12,10,4,21,22,26,33,11,2,24,3,27,29,23,14,30,31,20,15,25,8])
idx = np.argsort(gt)
print(idx + 1)
gt_mat = np.zeros((33, 33))
for i in range(33):
    gt_mat[i, idx[i]] = 1
    
    num_n=[i for i in np.logspace(1,6,6,base=2)]


rank_mat_1 = np.zeros_like(gt_mat)
rank_mat_2 = np.zeros_like(gt_mat)
min_error = np.inf
min_rank = None
min_n = None
for n in num_n:
#    dmap = diffusion_map.DiffusionMap.from_sklearn(n_evecs=2, alpha=0.5, epsilon='bgh', k=32) # DM
    dmap = Isomap(n_neighbors=5,n_components=2) 
#    dmap = LocallyLinearEmbedding(n_neighbors=5,n_components=2)
#    dmap = manifold.TSNE(n_components=2)
#    dmap = manifold.LocallyLinearEmbedding(n_neighbors=5, n_components=2, method='ltsa')
    ev = dmap.fit_transform(imgs)
    rank_1 = ev[:, 0].argsort()
    rank_2 = rank_1[::-1]

    for i in range(33):
        rank_mat_1[i, rank_1[i]] = 1
        rank_mat_2[i, rank_2[i]] = 1
    error_1 = np.linalg.norm(rank_mat_1 - gt_mat)
    error_2 = np.linalg.norm(rank_mat_2 - gt_mat)
    print('error_1:', error_1)
    print('error_2:', error_2)
    if error_1 < min_error:
        min_error = error_1
        min_rank = rank_1
        min_n = n
    if error_2 < min_error:
        min_error = error_2
        min_rank = rank_2
        min_n = n

print(min_rank)
print(min_error)
print(min_n)

plt.figure(figsize=[12, 5])
plt.scatter(ev[:, 0], ev[:, 1])
for i in range(33):
    plt.text(ev[i, 0] + 0.03, ev[i, 1] + 0.03, str(i + 1), fontsize=9)
plt.axis('off')
#plt.title('Diffusion map')
plt.show()



import matplotlib.pyplot as plt
import numpy as np


x = np.asarray([2,4,5,8,16,32]) 
y1 = np.asarray([6.782329983125268,3.4641016151377544,3.1622776601683795,3.7416573867739413,6.48074069840786,5.0990195135927845]) #ISOMAP
y2 = np.asarray([7.745966692414834,4.47213595499958,3.4641016151377544,5.477225575051661,6.782329983125268,6.324555320336759]) #LLE
y3 = np.asarray([7.745966692414834,7.483314773547883,2.8284271247461903,5.291502622129181,5.0990195135927845,6.324555320336759]) #LTSA

plt.plot(x,y1,color='mediumslateblue')
plt.plot(x,y2,color='salmon')
plt.plot(x,y3,color='mediumspringgreen')
plt.ylabel('Error in Frobenius Norm')             
plt.xlabel('The number of nearest neighbor')
plt.legend(['ISOMAP','LLE','LTSA'],bbox_to_anchor=(1.04, 1), loc=2, borderaxespad=0)
# for a, b in zip(x, y1):
#     plt.text(a, b, b, ha='center', va='bottom', fontsize=8)
# for a, b in zip(x, y2):
#     plt.text(a, b, b, ha='center', va='bottom', fontsize=8)
plt.show()

